# Drug Information RAG System using Vertex AI

## Overview
This notebook creates a Retrieval-Augmented Generation (RAG) system for drug information using:
- **Data Source**: Enriched Medicaid drug data from BigQuery
- **Embeddings**: Vertex AI Text Embeddings
- **Vector Store**: Vertex AI Vector Search (Matching Engine)
- **LLM**: Vertex AI PaLM 2 / Gemini

## Use Cases
- Ask questions about specific drugs
- Compare drug prices and forms
- Get recommendations based on drug characteristics
- Analyze drug pricing trends

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install --upgrade google-cloud-aiplatform \
    google-cloud-bigquery \
    langchain \
    langchain-google-vertexai \
    chromadb \
    pandas \
    numpy \
    tqdm

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Any
from tqdm import tqdm

# Google Cloud
from google.cloud import bigquery
from google.cloud import aiplatform
import vertexai
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel
from vertexai.preview.generative_models import GenerativeModel

# LangChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("✓ All packages imported successfully")

## 2. Configure GCP and Vertex AI

In [ ]:
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Any
from google.cloud import bigquery
from google.cloud import aiplatform
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
import faiss
import pickle
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration
PROJECT_ID = "gcp-project-deliverable"
LOCATION = "us-central1"
DATASET_ID = "medicaid_enriched"
TABLE_ID = "nadac_drugs_enriched"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print(f"✓ Vertex AI initialized for project: {PROJECT_ID}")
print(f"✓ Location: {LOCATION}")

## 3. Load Enriched Drug Data from BigQuery

In [ ]:
# Initialize BigQuery client
bq_client = bigquery.Client(project=PROJECT_ID)

# Query to get enriched drug data
query = f"""
SELECT 
    ndc,
    ndc_description,
    drug_name,
    drug_strength,
    drug_dosage,
    drug_form,
    CAST(nadac_per_unit AS FLOAT64) as nadac_per_unit,
    pricing_unit,
    pharmacy_type_indicator,
    otc,
    explanation_code,
    explanation_code_description,
    classification_for_rate_setting,
    CAST(corresponding_generic_drug_nadac_per_unit AS FLOAT64) as corresponding_generic_drug_nadac_per_unit,
    corresponding_generic_drug_effective_date,
    as_of_date,
    effective_date
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE ndc_description IS NOT NULL
    AND drug_name IS NOT NULL
LIMIT 10000
"""

print("Loading data from BigQuery...")
df = bq_client.query(query).to_dataframe()

print(f"\n✓ Loaded {len(df):,} drug records")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Load drug data (remove limit to process all records)
# For production: load all data
# For testing: add limit parameter, e.g., limit=1000
df = load_drug_data(PROJECT_ID, DATASET_ID, TABLE_ID, limit=None)

# Display sample data
print("\nSample drug data:")
df.head()

# Data exploration
print("=== Data Summary ===")
print(f"Total records: {len(df):,}")
print(f"Unique drugs: {df['drug_name'].nunique():,}")
print(f"Unique NDCs: {df['ndc'].nunique():,}")
print(f"\nDrug forms: {df['drug_form'].value_counts().head()}")
print(f"\nPrice range: ${df['nadac_per_unit'].min():.2f} - ${df['nadac_per_unit'].max():.2f}")

## 4. Create Drug Documents for RAG

In [ ]:
def create_drug_document(row: pd.Series) -> str:
    """
    Create a rich text document for each drug record.
    This will be embedded and stored in the vector database.
    """
    doc_parts = []
    
    # Basic information
    doc_parts.append(f"Drug Name: {row['drug_name']}")
    doc_parts.append(f"NDC Code: {row['ndc']}")
    
    # Formulation details
    if pd.notna(row['drug_strength']):
        doc_parts.append(f"Strength: {row['drug_strength']}")
    if pd.notna(row['drug_form']):
        doc_parts.append(f"Form: {row['drug_form']}")
    if pd.notna(row['drug_route']):
        doc_parts.append(f"Route of Administration: {row['drug_route']}")
    
    # Pricing information
    if pd.notna(row['nadac_per_unit']):
        try:
            price = float(row['nadac_per_unit'])
            doc_parts.append(f"Price per Unit: ${price:.2f}")
        except (ValueError, TypeError):
            pass  # Skip if conversion fails
    if pd.notna(row['pricing_unit']):
        doc_parts.append(f"Pricing Unit: {row['pricing_unit']}")
    if pd.notna(row['effective_date']):
        doc_parts.append(f"Effective Date: {row['effective_date']}")
    
    # Classification
    if pd.notna(row['classification_for_rate_setting']):
        classification_map = {
            'B': 'Brand Name',
            'G': 'Generic'
        }
        classification = classification_map.get(row['classification_for_rate_setting'], row['classification_for_rate_setting'])
        doc_parts.append(f"Classification: {classification}")
    
    # OTC indicator
    if pd.notna(row['otc']):
        otc_text = "Over-the-counter" if row['otc'] == 'Y' else "Prescription"
        doc_parts.append(f"Type: {otc_text}")
    
    # Pharmacy type
    if pd.notna(row['pharmacy_type_indicator']):
        pharmacy_types = {
            'C': 'Community pharmacy',
            'I': 'Institutional pharmacy',
            'B': 'Both community and institutional'
        }
        doc_parts.append(f"Available at: {pharmacy_types.get(row['pharmacy_type_indicator'], row['pharmacy_type_indicator'])}")
    
    # Explanation codes
    if pd.notna(row['explanation_code']):
        doc_parts.append(f"Explanation Code: {row['explanation_code']}")
    if pd.notna(row['explanation_code_description']):
        doc_parts.append(f"Note: {row['explanation_code_description']}")
    
    # Generic drug information
    if pd.notna(row['corresponding_generic_drug_nadac_per_unit']):
        doc_parts.append(f"Generic Drug Price: ${row['corresponding_generic_drug_nadac_per_unit']:.2f}")
    
    return "\n".join(doc_parts)

# Create documents
print("Creating drug documents...")
documents = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing drugs"):
    doc_text = create_drug_document(row)
    
    # Create LangChain Document with metadata
    doc = Document(
        page_content=doc_text,
        metadata={
            "ndc": str(row['ndc']),
            "drug_name": str(row['drug_name']),
            "drug_form": str(row['drug_form']) if pd.notna(row['drug_form']) else "",
            "price": float(row['nadac_per_unit']) if pd.notna(row['nadac_per_unit']) else 0.0,
            "effective_date": str(row['effective_date']) if pd.notna(row['effective_date']) else "",
            "source": "medicaid_enriched_data"
        }
    )
    documents.append(doc)

print(f"\n✓ Created {len(documents):,} drug documents")
print(f"\nExample document:")
print(documents[0].page_content[:500])

## 5. Create Embeddings and Vector Store

In [ ]:
# Initialize Vertex AI Embeddings
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@003",
    project=PROJECT_ID
)

print("✓ Initialized Vertex AI Embeddings (textembedding-gecko@003)")

In [ ]:
# Create vector store with ChromaDB
print("Creating vector store...")
print("This may take several minutes depending on the number of documents...")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="drug_information",
    persist_directory="./drug_vectorstore"
)

print(f"\n✓ Vector store created with {len(documents):,} documents")
print(f"✓ Persisted to: ./drug_vectorstore")

In [ ]:
# Test similarity search
test_query = "What is the price of ibuprofen tablets?"
results = vectorstore.similarity_search(test_query, k=3)

print(f"Test Query: {test_query}\n")
print("Top 3 similar documents:\n")
for i, doc in enumerate(results, 1):
    print(f"--- Result {i} ---")
    print(f"Drug: {doc.metadata['drug_name']}")
    print(f"Price: ${doc.metadata['price']:.2f}")
    print(f"Content: {doc.page_content[:200]}...\n")

## 6. Initialize LLM and Create RAG Chain

In [ ]:
# Initialize Vertex AI LLM (Gemini Pro)
llm = VertexAI(
    model_name="gemini-pro",
    project=PROJECT_ID,
    location=LOCATION,
    max_output_tokens=1024,
    temperature=0.2,  # Lower temperature for more factual responses
)

print("✓ Initialized Gemini Pro LLM")

In [ ]:
# Create custom prompt template
prompt_template = """You are a helpful assistant that answers questions about Medicaid drug pricing and information.

Use the following pieces of context from the Medicaid enriched drug database to answer the question.
If you don't know the answer based on the context provided, say "I don't have enough information to answer that question."
Always cite specific drug names, NDC codes, and prices when available.

Context:
{context}

Question: {question}

Answer (be specific and include relevant details like drug name, form, strength, and pricing):"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

print("✓ Created custom prompt template")

In [ ]:
# Create RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}  # Retrieve top 5 most relevant documents
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

print("✓ RAG chain created successfully")
print("✓ Retriever configured to fetch top 5 relevant documents")

## 7. Query the RAG System

In [ ]:
def ask_drug_question(question: str, show_sources: bool = True) -> None:
    """
    Ask a question to the drug RAG system and display the answer.
    
    Args:
        question: The question to ask
        show_sources: Whether to show source documents
    """
    print(f"\n{'='*80}")
    print(f"QUESTION: {question}")
    print(f"{'='*80}\n")
    
    # Get answer
    result = qa_chain({"query": question})
    
    # Display answer
    print("ANSWER:")
    print(result['result'])
    
    # Display sources
    if show_sources and 'source_documents' in result:
        print(f"\n{'-'*80}")
        print("SOURCE DOCUMENTS:")
        print(f"{'-'*80}\n")
        
        for i, doc in enumerate(result['source_documents'], 1):
            print(f"Source {i}:")
            print(f"  Drug: {doc.metadata['drug_name']}")
            print(f"  NDC: {doc.metadata['ndc']}")
            print(f"  Form: {doc.metadata['drug_form']}")
            print(f"  Price: ${doc.metadata['price']:.2f}")
            print(f"  Effective Date: {doc.metadata['effective_date']}")
            print()

### Example Questions

In [ ]:
# Question 1: Specific drug pricing
ask_drug_question("What is the price of ibuprofen 200mg tablets?")

In [ ]:
# Question 2: Drug comparison
ask_drug_question("Compare the prices of different forms of acetaminophen")

In [ ]:
# Question 3: Generic vs brand
ask_drug_question("What are the generic alternatives for expensive drugs?")

In [ ]:
# Question 4: Drug forms
ask_drug_question("What are the available forms of amoxicillin and their prices?")

In [ ]:
# Question 5: Specific NDC
ask_drug_question("Tell me about the drug with NDC 00378-1805-01")

## 8. Interactive Query Interface

In [ ]:
# Interactive mode - ask your own questions
print("\n" + "="*80)
print("INTERACTIVE DRUG QUERY SYSTEM")
print("="*80)
print("\nAsk questions about drug pricing, forms, strengths, and alternatives.")
print("Type 'quit' or 'exit' to stop.\n")

while True:
    question = input("\nYour question: ").strip()
    
    if question.lower() in ['quit', 'exit', 'q']:
        print("\nGoodbye!")
        break
    
    if not question:
        continue
    
    ask_drug_question(question, show_sources=True)

## 9. Advanced Features: Filtered Search

In [ ]:
def search_drugs_by_criteria(
    drug_name: str = None,
    drug_form: str = None,
    max_price: float = None,
    min_price: float = None,
    k: int = 5
) -> List[Document]:
    """
    Search for drugs with specific criteria.
    """
    # Build search query
    query_parts = []
    if drug_name:
        query_parts.append(f"Drug name: {drug_name}")
    if drug_form:
        query_parts.append(f"Form: {drug_form}")
    if max_price:
        query_parts.append(f"Price under ${max_price}")
    if min_price:
        query_parts.append(f"Price over ${min_price}")
    
    query = " ".join(query_parts) if query_parts else "all drugs"
    
    # Search
    results = vectorstore.similarity_search(query, k=k)
    
    # Filter by price if specified
    if max_price or min_price:
        filtered_results = []
        for doc in results:
            price = doc.metadata.get('price', 0)
            if max_price and price > max_price:
                continue
            if min_price and price < min_price:
                continue
            filtered_results.append(doc)
        results = filtered_results
    
    return results

# Example: Find ibuprofen tablets under $1
results = search_drugs_by_criteria(
    drug_name="ibuprofen",
    drug_form="tablet",
    max_price=1.0,
    k=10
)

print(f"Found {len(results)} results:\n")
for doc in results:
    print(f"- {doc.metadata['drug_name']} ({doc.metadata['drug_form']}): ${doc.metadata['price']:.2f}")

## 10. Save and Load Vector Store

In [ ]:
# Save vector store (already persisted)
print("Vector store is automatically persisted to: ./drug_vectorstore")

# To load it later:
# vectorstore = Chroma(
#     collection_name="drug_information",
#     persist_directory="./drug_vectorstore",
#     embedding_function=embeddings
# )

## 11. Export RAG System as Python Module

In [ ]:
# Create a standalone Python file for the RAG system
rag_module_code = '''
"""Drug Information RAG System"""

import vertexai
from langchain.vectorstores import Chroma
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

class DrugRAG:
    def __init__(self, project_id: str, location: str = "us-central1", 
                 vectorstore_path: str = "./drug_vectorstore"):
        # Initialize Vertex AI
        vertexai.init(project=project_id, location=location)
        
        # Load embeddings
        self.embeddings = VertexAIEmbeddings(
            model_name="textembedding-gecko@003",
            project=project_id
        )
        
        # Load vector store
        self.vectorstore = Chroma(
            collection_name="drug_information",
            persist_directory=vectorstore_path,
            embedding_function=self.embeddings
        )
        
        # Initialize LLM
        self.llm = VertexAI(
            model_name="gemini-pro",
            project=project_id,
            location=location,
            max_output_tokens=1024,
            temperature=0.2
        )
        
        # Create prompt
        prompt_template = """You are a helpful assistant for Medicaid drug information.
        
        Context: {context}
        Question: {question}
        
        Answer:"""
        
        PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )
        
        # Create chain
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 5}),
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )
    
    def ask(self, question: str) -> dict:
        """Ask a question and get an answer with sources."""
        return self.qa_chain({"query": question})
    
    def search(self, query: str, k: int = 5):
        """Search for similar drug documents."""
        return self.vectorstore.similarity_search(query, k=k)

# Usage:
# rag = DrugRAG(project_id="gcp-project-deliverable")
# result = rag.ask("What is the price of ibuprofen?")
# print(result['result'])
'''

# Save to file
with open('drug_rag_system.py', 'w') as f:
    f.write(rag_module_code)

print("✓ Saved RAG system to: drug_rag_system.py")

## 12. Performance Metrics

In [ ]:
# Test multiple queries and measure performance
import time

test_questions = [
    "What is the price of aspirin?",
    "What are the different forms of insulin?",
    "Compare generic and brand drug prices",
    "What drugs are available for pain relief?",
    "What is the most expensive drug in the database?"
]

print("Testing RAG system performance...\n")
total_time = 0

for i, question in enumerate(test_questions, 1):
    start_time = time.time()
    result = qa_chain({"query": question})
    elapsed = time.time() - start_time
    total_time += elapsed
    
    print(f"{i}. {question}")
    print(f"   Time: {elapsed:.2f}s")
    print(f"   Answer length: {len(result['result'])} chars")
    print(f"   Sources: {len(result['source_documents'])}\n")

print(f"\nAverage response time: {total_time/len(test_questions):.2f}s")

## 13. Next Steps and Enhancements

### Possible Improvements:

1. **Deploy to Production**:
   - Create Flask/FastAPI endpoint
   - Deploy to Cloud Run or GKE
   - Add authentication

2. **Enhanced Features**:
   - Add drug interaction checking
   - Include historical price trends
   - Add contraindications and warnings
   - Multi-drug comparison

3. **Better Embeddings**:
   - Fine-tune embeddings on drug data
   - Use Vertex AI Matching Engine for scale
   - Add metadata filtering

4. **UI Integration**:
   - Add to Streamlit dashboard
   - Create chatbot interface
   - Add voice interaction

5. **Data Updates**:
   - Schedule regular BigQuery syncs
   - Incremental vector store updates
   - Version control for embeddings

## Summary

This notebook created a complete RAG system for drug information:

✅ **Data Loading**: Extracted enriched drug data from BigQuery  
✅ **Document Creation**: Converted drug records into rich text documents  
✅ **Embeddings**: Used Vertex AI text embeddings  
✅ **Vector Store**: Stored in ChromaDB for similarity search  
✅ **LLM Integration**: Connected to Gemini Pro via Vertex AI  
✅ **RAG Chain**: Built LangChain RetrievalQA pipeline  
✅ **Interactive Queries**: Enabled Q&A about drug information  
✅ **Source Attribution**: Returns relevant source documents  
✅ **Reusable Module**: Exported as Python class  

**The system can now answer questions about drug pricing, forms, strengths, and alternatives using your enriched Medicaid data!**